In [8]:
!pip3 install pycaret
!pip3 install yahoofinancials
!pip3 install numpy
!pip3 install pandas
!pip3 install datetime
!pip3 install matplotlib
!pip3 install yahoofinancials
!pip3 install openpyxl

In [9]:
#Importing Libraries
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

In [10]:
ticker_details = pd.read_excel("Ticker List.xlsx")
ticker = ticker_details['Ticker'].to_list()
names = ticker_details['Description'].to_list()

In [11]:
#Preparing Date Range
end_date = datetime.strftime(datetime.today(), '%Y-%m-%d')
start_date = "2019-01-01"
date_range = pd.bdate_range(start=start_date, end=end_date)
values = pd.DataFrame({'Date': date_range})
values['Date'] = pd.to_datetime(values['Date'])


In [12]:
#Extracting Data from Yahoo Finance and Adding them to Values table using date as key
for i in ticker:
	raw_data = YahooFinancials(i)
	raw_data = raw_data.get_historical_price_data(start_date, end_date, "daily")
	df = pd.DataFrame(raw_data[i]['prices'])[['formatted_date', 'adjclose']]
	df.columns = ['Date1', i]
	df['Date1'] = pd.to_datetime(df['Date1'])
	values = values.merge(df, how='left', left_on='Date', right_on='Date1')
	values = values.drop(labels='Date1', axis=1)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#Renaming columns to represent instrument names rather than their ticker codes for ease of readability
names.insert(0, 'Date')
values.columns = names

#Front filling the NaN values in the data set
values = values.fillna(method="ffill", axis=0)
values = values.fillna(method="bfill", axis=0)

# Co-ercing numeric type to all columns except Date
cols = values.columns.drop('Date')
values[cols] = values[cols].apply(pd.to_numeric, errors='coerce').round(decimals=1)
imp = ['Gold', 'Silver', 'Crude Oil', 'S&P500', 'MSCI EM ETF']

# Calculating Short term -Historical Returns
change_days = [1, 3, 5, 14, 21]

data = pd.DataFrame(data=values['Date'])
for i in change_days:
	x = values[cols].pct_change(periods=i).add_suffix("-T-" + str(i))
	data = pd.concat(objs=(data, x), axis=1)
	x = []

# Calculating Long term Historical Returns
change_days = [60, 90, 180, 250]

for i in change_days:
	x = values[imp].pct_change(periods=i).add_suffix("-T-" + str(i))
	data = pd.concat(objs=(data, x), axis=1)
	x = []

#Calculating Moving averages for Gold
moving_avg = pd.DataFrame(values['Date'], columns=['Date'])
moving_avg['Date'] = pd.to_datetime(moving_avg['Date'], format='%Y-%b-%d')
moving_avg['Gold/15SMA'] = (values['Gold'] / (values['Gold'].rolling(window=15).mean())) - 1
moving_avg['Gold/30SMA'] = (values['Gold'] / (values['Gold'].rolling(window=30).mean())) - 1
moving_avg['Gold/60SMA'] = (values['Gold'] / (values['Gold'].rolling(window=60).mean())) - 1
moving_avg['Gold/90SMA'] = (values['Gold'] / (values['Gold'].rolling(window=90).mean())) - 1
moving_avg['Gold/180SMA'] = (values['Gold'] / (values['Gold'].rolling(window=180).mean())) - 1
moving_avg['Gold/90EMA'] = (values['Gold'] / (values['Gold'].ewm(span=90, adjust=True, ignore_na=True).mean())) - 1
moving_avg['Gold/180EMA'] = (values['Gold'] / (values['Gold'].ewm(span=180, adjust=True, ignore_na=True).mean())) - 1
moving_avg = moving_avg.dropna(axis=0)

#Merging Moving Average values to the feature space

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%b-%d')
data = pd.merge(left=data, right=moving_avg, how='left', on='Date')
data = data[data['Gold-T-250'].notna()]
prediction_data = data.copy()

In [ ]:
from pycaret.classification import *

#Loading the stored model
classifier_22 = load_model("22D Classifier");

#Making Predictions
prediction = predict_model(classifier_22, data=prediction_data)


In [ ]:
prediction[['Date', 'Label', 'Score']].tail(10)